# **Crypto On-chain Data Analysis**
This notebook analyzes historical tokenomics for major cryptocurrencies (BTC, ETH, BNB, SOL) using data from CoinGecko API.

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pycoingecko import CoinGeckoAPI

# Initialize CoinGecko API client
cg = CoinGeckoAPI()

# Set default template for plotly
import plotly.io as pio
pio.templates.default = "plotly_dark"


In [5]:
# Define the cryptocurrencies we want to analyze
crypto = pd.DataFrame({
    'symbol': ['BTC', 'ETH', 'BNB', 'SOL', 'XRP', 'AVAX'],
    'name': ['Bitcoin', 'Ethereum', 'Binance Coin', 'Solana', 'XRP', 'Avalanche'],
    'coingecko_id': ['bitcoin', 'ethereum', 'binancecoin', 'solana', 'ripple', 'avalanche-2']
})

print("Cryptocurrencies to analyze:")
print(crypto)

Cryptocurrencies to analyze:
  symbol          name coingecko_id
0    BTC       Bitcoin      bitcoin
1    ETH      Ethereum     ethereum
2    BNB  Binance Coin  binancecoin
3    SOL        Solana       solana
4    XRP           XRP       ripple
5   AVAX     Avalanche  avalanche-2


In [6]:
# Fetching Tokenomics Data
tokenomics_data = []

for _, row in crypto.iterrows():
    try:
        print(f"Fetching data for {row['name']}...")
        # Fetch detailed coin data from CoinGecko
        coin_data = cg.get_coin_by_id(row['coingecko_id'])
        data = {
            'symbol': row['symbol'],
            'current_price': coin_data['market_data']['current_price']['usd'],
            'market_cap': coin_data['market_data']['market_cap']['usd'],
            'total_volume': coin_data['market_data']['total_volume']['usd'],
            'circulating_supply': coin_data['market_data']['circulating_supply'],
            'total_supply': coin_data['market_data']['total_supply'],
            'max_supply': coin_data['market_data']['max_supply'],
            'fdv': coin_data['market_data'].get('fully_diluted_valuation', {}).get('usd'),
        }
        tokenomics_data.append(data)
        
    except Exception as e:
        print(f"Error fetching data for {row['symbol']}: {e}")

# Convert to DataFrame
tokenomics_df = pd.DataFrame(tokenomics_data)

Fetching data for Bitcoin...
Fetching data for Ethereum...
Fetching data for Binance Coin...
Fetching data for Solana...
Fetching data for XRP...
Fetching data for Avalanche...


In [ ]:
# 1. Market Cap Comparison (Bar Chart)
fig1 = px.bar(
    tokenomics_df,
    x='symbol',
    y='market_cap',
    title='Market Capitalization by Cryptocurrency',
    labels={'market_cap': 'Market Cap (USD)', 'symbol': 'Cryptocurrency'},
    color='symbol',
    color_discrete_map={
        'BTC': '#F7931A',  # Bitcoin Orange
        'ETH': '#636BFF',  # Ethereum Blue
        'BNB': '#F3BA2F',  # Binance Yellow
        'SOL': '#CF62E7',   # Solana Pink
        'XRP': '#4B8BBE',  # XRP Blue
        'AVAX': '#E84142'  # Avalanche Red
    },
)
fig1.show()

# 2. Supply Metrics Comparison (Grouped Bar Chart)
supply_data = pd.melt(
    tokenomics_df,
    id_vars=['symbol'],
    value_vars=['circulating_supply', 'total_supply', 'max_supply'],
    var_name='supply_type',
    value_name='supply',
)
fig2 = px.bar(
    supply_data,
    x='symbol',
    y='supply',
    color='supply_type',
    title='Supply Metrics by Cryptocurrency',
    barmode='group',
    labels={'supply': 'Supply Amount', 'symbol': 'Cryptocurrency'}
)
fig2.show()

# 3. Market Cap vs Volume (Scatter Plot)
fig3 = px.scatter(
    tokenomics_df,
    x='market_cap',
    y='total_volume',
    text='symbol',
    title='Market Cap vs Trading Volume',
    labels={
        'market_cap': 'Market Cap (USD)',
        'total_volume': 'Trading Volume (USD)'
    },
    size='market_cap',
    log_x=True,
    log_y=True
)
fig3.show()

# 4. Price vs Supply (Bubble Chart)
fig4 = px.scatter(
    tokenomics_df,
    x='circulating_supply',
    y='current_price',
    size='market_cap',
    text='symbol',
    title='Price vs Circulating Supply (Bubble size = Market Cap)',
    labels={
        'circulating_supply': 'Circulating Supply',
        'current_price': 'Price (USD)',
    },
    log_x=True,
    log_y=True
)
fig4.show()

# 5. Market Dominance (Pie Chart)
fig5 = px.pie(
    tokenomics_df,
    values='market_cap',
    names='symbol',
    title='Market Cap Distribution'
)
fig5.show()